<a href="https://colab.research.google.com/github/Madhu3456/IITISoC-22-AI-ML-11-Song-Recommendation/blob/main/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import keras
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import warnings
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LeakyReLU


Read data from G-drive

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fer2013.csv")
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


To check if the dataset is empty

In [5]:
df.isnull().sum()

emotion    0
pixels     0
Usage      0
dtype: int64

*Data PreProcessing*

Shuffle data

In [6]:
df = df.sample(frac=1)

Changing image pixels to Numpy array

In [7]:
train_pixels = df["pixels"].astype(str).str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)

Standardization

In [8]:
pixels = train_pixels.reshape((35887*2304,1))

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pixels = scaler.fit_transform(pixels)

In [10]:
pixels = train_pixels.reshape((35887, 48, 48,1))

In [11]:
df['pixels']

15722    255 255 255 255 255 255 255 253 255 158 93 105...
29994    125 131 137 146 149 149 153 155 153 155 157 15...
28954    110 118 112 107 106 92 95 102 100 105 105 91 5...
19398    22 22 22 22 23 24 25 26 44 52 61 82 96 105 116...
18158    195 141 167 210 203 205 205 206 206 206 205 20...
                               ...                        
20670    57 66 119 174 177 186 193 191 193 196 201 204 ...
4322     40 47 52 68 91 113 127 135 158 173 179 182 189...
18963    99 89 73 62 42 33 22 17 10 9 17 21 27 36 42 49...
8260     190 186 182 180 168 147 110 133 120 69 63 93 7...
28097    148 150 138 138 162 146 141 128 80 57 48 43 75...
Name: pixels, Length: 35887, dtype: object

Illumination Normalisation

In [12]:
def normalize(img):
  lmin = float(img.min())
  lmax = float(img.max())
  return np.floor((img-lmin)/(lmax-lmin*255.0))

  for i in range(0,35887):
    df['pixels'][i] = normalize(df['pixels'][i] )

Test Train data split

One Hot Encoding

In [13]:
labels = to_categorical(df['emotion'], num_classes=7)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(pixels,labels, test_size=0.1,shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1,shuffle=False)

Data Augumentation

In [14]:
datagen = ImageDataGenerator(  width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2)
valgen = ImageDataGenerator(   width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2) 

In [16]:
datagen.fit(X_train)
valgen.fit(X_val)

train_generator = datagen.flow(X_train, y_train, batch_size=64)
val_generator = datagen.flow(X_val, y_val, batch_size=64)



Model

In [17]:
_input = Input((48,48,1)) 

In [18]:
conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(256, activation="relu")(flat)
dense2 = Dense(128, activation="relu")(dense1)
dense3 = Dense(64, activation="relu")(dense2)
output = Dense(7, activation="softmax")(dense3)

model  = Model(inputs=_input, outputs=output)

In [19]:
model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       147584    
                                                             

Early stopping to prevent overfitting

In [21]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, 
                              restore_best_weights=True,mode="max",patience = 5),
                ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")]


Fit model

In [22]:
history = model.fit(train_generator,
                    epochs=30,
                    batch_size=64,   
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=val_generator)

Epoch 1/30
455/455 [==============================] - ETA: 0s - loss: 1.8092 - accuracy: 0.2576
Epoch 1: val_accuracy improved from -inf to 0.31362, saving model to best_model.h5
455/455 [==============================] - 42s 65ms/step - loss: 1.8092 - accuracy: 0.2576 - val_loss: 1.7252 - val_accuracy: 0.3136
Epoch 2/30
455/455 [==============================] - ETA: 0s - loss: 1.5669 - accuracy: 0.3798
Epoch 2: val_accuracy improved from 0.31362 to 0.41115, saving model to best_model.h5
455/455 [==============================] - 29s 63ms/step - loss: 1.5669 - accuracy: 0.3798 - val_loss: 1.4953 - val_accuracy: 0.4111
Epoch 3/30
455/455 [==============================] - ETA: 0s - loss: 1.4154 - accuracy: 0.4462
Epoch 3: val_accuracy improved from 0.41115 to 0.46409, saving model to best_model.h5
455/455 [==============================] - 30s 66ms/step - loss: 1.4154 - accuracy: 0.4462 - val_loss: 1.3762 - val_accuracy: 0.4641
Epoch 4/30
455/455 [==============================] - ETA:

In [24]:
model.save('model_30epoch_Adam.h5')

In [25]:
model = tf.keras.models.load_model('model_30epoch_Adam.h5')

In [26]:
loss = model.evaluate(X_test,y_test) 
print("Test Acc: " + str(loss[1]))

113/113 [==============================] - 3s 18ms/step - loss: 0.9505 - accuracy: 0.6537
Test Acc: 0.6536639928817749
